In [1]:
import hls4ml
print("hls4ml version:", hls4ml.__version__)


hls4ml version: 0.9.0.dev5+g033d4382


In [2]:
import numpy as np
import pandas as pd
import keras
from keras import layers
import tensorflow as tf
import qkeras
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Activation
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
#from callbacks import all_callbacks

In [3]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
trim = 4
x_train_ = np.round(x_train[:, trim:28-trim, trim:28-trim].reshape(x_train.shape[0],-1))
x_test_ = np.round(x_test[:, trim:28-trim, trim:28-trim].reshape(x_test.shape[0],-1))
input_shape = x_train_.shape[1:]
print(input_shape)


(400,)


In [5]:
x_train_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
x_train_.shape, x_test_.shape

((60000, 400), (10000, 400))

In [30]:
model = Sequential()
model.add(keras.Input(shape=input_shape, name="input0"))


model.add(
    QDense(
        10,
        name='fc1',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu1'))
model.add(
    QDense(
        10,
        name='fc2',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(QActivation(activation=quantized_relu(6), name='relu2'))
model.add(
    QDense(
        num_classes,
        name='fc3',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
model.add(Activation(activation='softmax', name='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (QDense)                (None, 10)                4010      
                                                                 
 relu1 (QActivation)         (None, 10)                0         
                                                                 
 fc2 (QDense)                (None, 10)                110       
                                                                 
 relu2 (QActivation)         (None, 10)                0         
                                                                 
 fc3 (QDense)                (None, 10)                110       
                                                                 
 softmax (Activation)        (None, 10)                0         
                                                                 
Total params: 4230 (16.52 KB)
Trainable params: 4230 (

In [31]:
batch_size = 128
epochs = 15

model.compile(optimizer="adam", loss=['categorical_crossentropy'], metrics=['accuracy'])
model.fit(x_train_, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 1s 2ms/step - loss: 1.5562 - accuracy: 0.6201 - val_loss: 1.0976 - val_accuracy: 0.7970
Epoch 2/15
422/422 [==============================] - 0s 1ms/step - loss: 0.9417 - accuracy: 0.8202 - val_loss: 0.7315 - val_accuracy: 0.8848
Epoch 3/15
422/422 [==============================] - 0s 1ms/step - loss: 0.7002 - accuracy: 0.8688 - val_loss: 0.5813 - val_accuracy: 0.9000
Epoch 4/15
422/422 [==============================] - 0s 1ms/step - loss: 0.5969 - accuracy: 0.8833 - val_loss: 0.5143 - val_accuracy: 0.9087
Epoch 5/15
422/422 [==============================] - 0s 1ms/step - loss: 0.5471 - accuracy: 0.8889 - val_loss: 0.4766 - val_accuracy: 0.9095
Epoch 6/15
422/422 [==============================] - 0s 1ms/step - loss: 0.5191 - accuracy: 0.8923 - val_loss: 0.4605 - val_accuracy: 0.9123
Epoch 7/15
422/422 [==============================] - 0s 1ms/step - loss: 0.5062 - accuracy: 0.8954 - val_loss: 0.4568 - val_accuracy: 0.9117
Epoch 

In [32]:
score = model.evaluate(x_test_, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

save_model(model, "qModel.keras")

Test loss: 0.47468385100364685
Test accuracy: 0.8988000154495239


In [37]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
config['LayerName']['input0']['Precision'] = {'result': 'ap_uint<1>'}  # Set input precision to 1-bit
print("-----------------------------------")
print(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='model_1/hls4ml_prj',
    part='xcu250-figd2104-2L-e', backend='Vitis'
)
hls_model.compile()

Interpreting Sequential
Topology:
Layer name: input0, layer type: InputLayer, input shapes: [[None, 400]], output shape: [None, 400]
Layer name: fc1, layer type: QDense, input shapes: [[None, 400]], output shape: [None, 10]
Layer name: relu1, layer type: Activation, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: fc2, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: relu2, layer type: Activation, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: fc3, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: softmax, layer type: Softmax, input shapes: [[None, 10]], output shape: [None, 10]
-----------------------------------
{'Model': {'Precision': 'fixed<16,6>', 'ReuseFactor': 1, 'Strategy': 'Latency', 'BramFactor': 1000000000, 'TraceOutput': False}, 'LayerName': {'input0': {'Trace': False, 'Precision': {'result': 'ap_uint<1>'}}, 'fc1': {'Trace': False, 'Precision': {'result': 'fixed<16,6>', 

FileNotFoundError: Could not find module 'C:\Users\Tim\PycharmProjects\rs_final\model_1\hls4ml_prj\firmware\myproject-eC7Bc7Fa.so' (or one of its dependencies). Try using the full path with constructor syntax.

In [38]:
hls_model.write()

Writing HLS project
Done


In [41]:
import os
os.environ['PATH'] += os.pathsep + 'D:/ProgramFiles/Xilinx/Vitis_HLS/2023.2/bin'
hls_model.build(csim=0)

Vivado synthesis report not found.
Cosim report not found.
Timing report not found.


{'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '4.344',
  'BestLatency': '10',
  'WorstLatency': '10',
  'IntervalMin': '1',
  'IntervalMax': '1',
  'BRAM_18K': '12',
  'DSP': '10',
  'FF': '2629',
  'LUT': '61606',
  'URAM': '0',
  'AvailableBRAM_18K': '5376',
  'AvailableDSP': '12288',
  'AvailableFF': '3456000',
  'AvailableLUT': '1728000',
  'AvailableURAM': '1280'}}

python3: can't open file 'hls4ml': [Errno 2] No such file or directory
